In [47]:
import torch
import torch.nn
from torchvision.models import convnext_large, convnext_small
from torch.optim import SGD, Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import os

In [16]:
transform = transforms.Compose([
                                transforms.Resize([232], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([224,224]),
                                transforms.ToTensor(), # Rescaled to 0-1 aswell
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

target_transform = transforms.Compose([
                                transforms.Resize([232], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([224,224]),
                                transforms.ToTensor(),
])

In [17]:
class CustomDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.transform= transform
        self.target_transform = target_transform
        self.path = path
        self.labels = []
        self.images = []
        self.count = 0
        for i, direc in enumerate(os.listdir(self.path)[1:]): # [1:] because MacOS has .DS_Store at 0th index
            for j, file_name in enumerate(os.listdir(os.path.join(self.path,direc))):
                self.labels.append(i)
                image_path = os.path.join(self.path, direc ,file_name)
                image = Image.open(image_path)
                image = transform(image)
                self.images.append(image)
        
        # Early loading
        
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Lazy loading
        
#         directory_idx = idx//100
#         file_idx = idx%100
#         label = directory_idx

        return self.images[idx], self.labels[idx]
        

In [18]:
dataset = CustomDataset('tiny_FR', transform)

In [19]:
torch.manual_seed(10)
index = torch.randperm(500)

In [20]:
data_train = Subset(dataset,index[:400])
data_valid = Subset(dataset,index[400:])

In [21]:
torch.manual_seed(1)
data = DataLoader(data_train, 32, shuffle=True)
data_vl = DataLoader(data_valid, 4, shuffle=True)

In [22]:
next(iter(data))[0].shape

torch.Size([32, 3, 224, 224])

In [48]:
model = convnext_small(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /Users/muhammadwaseem/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth


  0%|          | 0.00/192M [00:00<?, ?B/s]

In [55]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [56]:
for param in model.parameters():
    param.requires_grad = False

In [57]:
num_ftrs = model.classifier[2].in_features
model.classifier[2] = torch.nn.Linear(num_ftrs, 5)

In [58]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [59]:
model = model.to('mps')

In [60]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier[2].parameters(), lr=0.01, momentum=0.9)

In [61]:
def train(epoch, data, data_vl):
    train_loss = [0] * epoch
    train_accuracy = [0] * epoch
    
    valid_loss = [0] * epoch
    valid_accuracy = [0] * epoch
    model.train()
    for i in range(epoch):
        for x_batch, y_batch in data:
            #print(x_batch.shape)
            x_batch = x_batch.to('mps')
            y_batch = y_batch.to('mps')
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss[i] += loss.item() * x_batch.size(0)
            correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
            train_accuracy[i] += correct.to(torch.float32).item()
        train_loss[i] /= len(data.dataset)
        train_accuracy[i] /= len(data.dataset)
        
        with torch.no_grad():
            for x_batch, y_batch in data_vl:
                x_batch = x_batch.to('mps')
                y_batch = y_batch.to('mps')
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                valid_loss[i] += loss.item() * x_batch.size(0)
                correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
                valid_accuracy[i] += correct.item()
            valid_loss[i] /= len(data_vl.dataset)
            valid_accuracy[i] /= len(data_vl.dataset)
        print(f'Epoch {i+1} accuracy: {train_accuracy[i]:.4f} val_accuracy:{valid_accuracy[i]:.4f}')
        print(f'Epoch {i+1} loss: {train_loss[i]:.4f} val_loss:{valid_loss[i]:.4f}')
        print()
            

In [62]:
train(50, data, data_vl)

Epoch 1 accuracy: 0.5825 val_accuracy:0.8000
Epoch 1 loss: 1.0751 val_loss:0.5419

Epoch 2 accuracy: 0.8825 val_accuracy:0.8400
Epoch 2 loss: 0.3671 val_loss:0.3632

Epoch 3 accuracy: 0.9275 val_accuracy:0.9000
Epoch 3 loss: 0.2204 val_loss:0.3009

Epoch 4 accuracy: 0.9550 val_accuracy:0.9400
Epoch 4 loss: 0.1720 val_loss:0.2284

Epoch 5 accuracy: 0.9725 val_accuracy:0.9400
Epoch 5 loss: 0.1384 val_loss:0.2110

Epoch 6 accuracy: 0.9750 val_accuracy:0.9400
Epoch 6 loss: 0.1226 val_loss:0.1698

Epoch 7 accuracy: 0.9775 val_accuracy:0.9200
Epoch 7 loss: 0.1100 val_loss:0.2153

Epoch 8 accuracy: 0.9875 val_accuracy:0.9500
Epoch 8 loss: 0.0967 val_loss:0.1784

Epoch 9 accuracy: 0.9825 val_accuracy:0.9300
Epoch 9 loss: 0.0774 val_loss:0.2017

Epoch 10 accuracy: 0.9850 val_accuracy:0.9400
Epoch 10 loss: 0.0790 val_loss:0.2037

Epoch 11 accuracy: 0.9900 val_accuracy:0.9700
Epoch 11 loss: 0.0787 val_loss:0.1633

Epoch 12 accuracy: 0.9900 val_accuracy:0.9500
Epoch 12 loss: 0.0711 val_loss:0.1673

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix